## Feature implementation

In [ ]:
import spacy
from statistics import mean
import re

In [ ]:
train

,Filename,text,Language,Proficiency
0,88.txt,Some people might think that traveling in a gr...,KOR,high
1,278.txt,IThe importance and popularity of travelling i...,DEU,medium
2,348.txt,"It is an important decision, how to plan your ...",TUR,high
3,666.txt,Some people believe that young people can enjo...,ZHO,medium
4,733.txt,Travelling is usually considered as good recr...,TEL,medium
...,...,...,...,...
10995,1999007.txt,"Nowadays, more and more people go abroad,no ma...",ZHO,medium
10996,1999198.txt,\tIn accomplishing something that is risky com...,KOR,high
10997,1999363.txt,"At the beginning of the 21st century, the incr...",SPA,high
10998,1999931.txt,The number of cars in use across the world has...,HIN,high


In [ ]:
# create separate csv file
# do not run again
#train.to_csv("toefl11_trainingdata")
#train.to_csv('/content/drive/MyDrive/NLP Shared task/toefl11_trainingdata.csv')

In [ ]:
# load training data
toefl = pd.read_csv("toefl11_trainingdata")
try: 
    toefl = toefl.drop(['Unnamed: 0'], axis=1)
except KeyError:
    pass
toefl.head()

,Filename,text,Language,Proficiency
0,88.txt,Some people might think that traveling in a gr...,KOR,high
1,278.txt,IThe importance and popularity of travelling i...,DEU,medium
2,348.txt,"It is an important decision, how to plan your ...",TUR,high
3,666.txt,Some people believe that young people can enjo...,ZHO,medium
4,733.txt,Travelling is usually considered as good recr...,TEL,medium


### POS tagging

In [ ]:
spacy_model = spacy.load('en_core_web_sm', disable=('ner', 'parser'))
toefl['POS'] = toefl['text'].apply(lambda x: [y.pos_ for y in  spacy_model(x)])
toefl['POS'] = toefl['POS'].apply(' '.join)
toefl.head()

,Filename,text,Language,Proficiency,POS
0,88.txt,Some people might think that traveling in a gr...,KOR,high,DET NOUN VERB VERB SCONJ VERB ADP DET NOUN VER...
1,278.txt,IThe importance and popularity of travelling i...,DEU,medium,ADP NOUN CCONJ NOUN ADP VERB AUX ADV VERB PUNC...
2,348.txt,"It is an important decision, how to plan your ...",TUR,high,PRON AUX DET ADJ NOUN PUNCT ADV PART VERB DET ...
3,666.txt,Some people believe that young people can enjo...,ZHO,medium,DET NOUN VERB SCONJ ADJ NOUN VERB VERB NOUN AD...
4,733.txt,Travelling is usually considered as good recr...,TEL,medium,PROPN AUX SPACE ADV VERB SCONJ ADJ NOUN SPACE ...


#### Dependency parsing

In [ ]:
spacy_model2 = spacy.load("en_core_web_sm", disable=("ner"))
toefl['DEP'] = toefl['text'].apply(lambda x: [y.dep_ for y in  spacy_model2(x)])
toefl['DEP'] = toefl['DEP'].apply(' '.join)


In [ ]:
toefl.head()

,Filename,text,Language,Proficiency,POS,DEP
0,88.txt,Some people might think that traveling in a gr...,KOR,high,DET NOUN VERB VERB SCONJ VERB ADP DET NOUN VER...,det nsubj aux ROOT mark csubj prep det pobj ac...
1,278.txt,IThe importance and popularity of travelling i...,DEU,medium,ADP NOUN CCONJ NOUN ADP VERB AUX ADV VERB PUNC...,det nsubj cc conj prep pcomp aux advmod ROOT p...
2,348.txt,"It is an important decision, how to plan your ...",TUR,high,PRON AUX DET ADJ NOUN PUNCT ADV PART VERB DET ...,nsubj ROOT det amod attr punct advmod aux advc...
3,666.txt,Some people believe that young people can enjo...,ZHO,medium,DET NOUN VERB SCONJ ADJ NOUN VERB VERB NOUN AD...,det nsubj ROOT mark amod nsubj aux ccomp dobj ...
4,733.txt,Travelling is usually considered as good recr...,TEL,medium,PROPN AUX SPACE ADV VERB SCONJ ADJ NOUN SPACE ...,nsubjpass auxpass advmod ROOT prep amod pobj ...


#### NER

In [ ]:
spacy_model3 = spacy.load("en_core_web_sm")
toefl['NER'] = toefl['text'].apply(lambda x: list(spacy_model3(x).ents))

In [ ]:
toefl.head()

,Filename,text,Language,Proficiency,POS,DEP,NER
0,88.txt,Some people might think that traveling in a gr...,KOR,high,DET NOUN VERB VERB SCONJ VERB ADP DET NOUN VER...,det nsubj aux ROOT mark csubj prep det pobj ac...,"[(First), (Thailand), (first), (first), (Bangk..."
1,278.txt,IThe importance and popularity of travelling i...,DEU,medium,ADP NOUN CCONJ NOUN ADP VERB AUX ADV VERB PUNC...,det nsubj cc conj prep pcomp aux advmod ROOT p...,[(IThe)]
2,348.txt,"It is an important decision, how to plan your ...",TUR,high,PRON AUX DET ADJ NOUN PUNCT ADV PART VERB DET ...,nsubj ROOT det amod attr punct advmod aux advc...,"[(one), (one), (at, least, three), (one), (fir..."
3,666.txt,Some people believe that young people can enjo...,ZHO,medium,DET NOUN VERB SCONJ ADJ NOUN VERB VERB NOUN AD...,det nsubj ROOT mark amod nsubj aux ccomp dobj ...,"[(First), (One), (two, years), (today)]"
4,733.txt,Travelling is usually considered as good recr...,TEL,medium,PROPN AUX SPACE ADV VERB SCONJ ADJ NOUN SPACE ...,nsubjpass auxpass advmod ROOT prep amod pobj ...,"[(First), (Secondly), (India), (Disrespectful)..."


In [ ]:
# save tagged nli file
toefl.to_csv("/content/drive/MyDrive/NLP Shared task/toefl11_trainingdata_tagged.csv")

### 3. Add custom features

In [2]:
#imports
import os
import pandas as pd

import spacy
from statistics import mean
import re

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# load tagged nli file
pd.set_option('display.max_colwidth', 1)
toefl_tagged = pd.read_csv("/content/drive/MyDrive/NLP Shared task/toefl11_trainingdata_tagged.csv")
#try: 
   # toefl_tagged = toefl_tagged.drop(['Unnamed: 0'], axis=1)
#except KeyError:
    #pass

toefl_tagged.head()

,Unnamed: 0,Filename,text,Language,Proficiency,POS,DEP,NER
0,0,88.txt,"Some people might think that traveling in a group led by a tour guide is a good way. But, a group tour normally has its strict schedule and does not offer the tourists in group any chance to meet many people in new place. \nSo, I strongly disagree with the statement that the best way to travel is in a group led by a tour guide. \n\nFirst, a group tour has its own strict schedule and all the tour team members should follow it. And it means that I can not choose the place where I want go and the food which I want to eat. For example, when I went to Thailand for the first time, I chose a group tour because I was young and it was the first time to go abroad. I really wanted to go many attractive shops and spots in the street in Bangkok, but I couldn't go there because I was in a group tour and I had to follow the schedule which the tour company offered. And I had to wake up early in the morning and had to go back to hotel in early evening, even though I wanted to enjoy more sleep in the bed in the morning and wanted to go outside in the night. It was really bad experience. \n\nSecond, a group tour does not offer many opportunities to meet other people for its tour team members. Becuase a group tour moves together by its tour bus, it is difficult to meet many people in new places. But if I travel alone, I can meet many people and feel their cultures. When I went to Tokyo by myself, I met really many people there. I could talk with them and some of them became my friends! Even though I could not go a lot of sightseeing spots as efficiently as a group tour, choosing a place where I go and going there by a subway were really exciting experience and Japanese people were kind to let me know the ways. \n\nAll things considered, I definitely belive that a group tour led by a tour guide is not interesting or exciting at all. By going new place alone, we can learn more about the new place and feel much about the people there. \n",KOR,high,DET NOUN VERB VERB SCONJ VERB ADP DET NOUN VERB ADP DET NOUN NOUN AUX DET ADJ NOUN PUNCT CCONJ PUNCT DET NOUN NOUN ADV AUX DET ADJ NOUN CCONJ AUX PART VERB DET NOUN ADP NOUN DET NOUN PART VERB ADJ NOUN ADP ADJ NOUN PUNCT SPACE ADV PUNCT PRON ADV VERB ADP DET NOUN SCONJ DET ADJ NOUN PART VERB AUX ADP DET NOUN VERB ADP DET NOUN NOUN PUNCT SPACE ADV PUNCT DET NOUN NOUN AUX DET ADJ ADJ NOUN CCONJ DET DET NOUN NOUN NOUN VERB VERB PRON PUNCT CCONJ PRON VERB SCONJ PRON VERB PART VERB DET NOUN ADV PRON VERB VERB CCONJ DET NOUN DET PRON VERB PART VERB PUNCT ADP NOUN PUNCT ADV PRON VERB ADP PROPN ADP DET ADJ NOUN PUNCT PRON VERB DET NOUN NOUN SCONJ PRON AUX ADJ CCONJ PRON AUX DET ADJ NOUN PART VERB ADV PUNCT PRON ADV VERB PART VERB ADJ ADJ NOUN CCONJ NOUN ADP DET NOUN ADP PROPN PUNCT CCONJ PRON VERB PART VERB ADV SCONJ PRON AUX ADP DET NOUN NOUN CCONJ PRON AUX PART VERB DET NOUN DET DET NOUN NOUN VERB PUNCT CCONJ PRON AUX PART VERB ADP ADV ADP DET NOUN CCONJ AUX PART VERB ADV ADP NOUN ADP ADJ NOUN PUNCT ADV SCONJ PRON VERB PART VERB ADJ NOUN ADP DET NOUN ADP DET NOUN CCONJ VERB PART VERB ADV ADP DET NOUN PUNCT PRON AUX ADV ADJ NOUN PUNCT SPACE ADV PUNCT DET NOUN NOUN AUX PART VERB ADJ NOUN PART VERB ADJ NOUN ADP DET NOUN NOUN NOUN PUNCT NOUN DET NOUN NOUN VERB ADV ADP DET NOUN NOUN PUNCT PRON AUX ADJ PART VERB ADJ NOUN ADP ADJ NOUN PUNCT CCONJ SCONJ PRON VERB ADV PUNCT PRON VERB VERB ADJ NOUN CCONJ VERB DET NOUN PUNCT ADV PRON VERB ADP PROPN ADP PRON PUNCT PRON VERB ADV ADJ NOUN ADV PUNCT PRON VERB VERB ADP PRON CCONJ DET ADP PRON VERB DET NOUN PUNCT ADV SCONJ PRON VERB PART VERB DET NOUN ADP NOUN NOUN ADV ADV SCONJ DET NOUN NOUN PUNCT VERB DET NOUN ADV PRON VERB CCONJ VERB ADV ADP DET NOUN AUX ADV ADJ NOUN CCONJ ADJ NOUN AUX ADJ PART VERB PRON VERB DET NOUN PUNCT SPACE DET NOUN VERB PUNCT PRON ADV VERB SCONJ DET NOUN NOUN VERB ADP DET NOUN NOUN AUX PART ADJ CCONJ ADJ ADV ADV PUNCT ADP VERB ADJ NOUN ADV PUNCT PRON VERB VERB ADJ ADP DET ADJ NOUN C

#### Proportion of punctuation marks

In [6]:
# create function
def prop_punct(series):
    """Calculates the proportion of the number of punctuation marks compared to the total of POS tags"""
    
    total_pos = len(series.split())
    punct_count = 0
    for tag in series.split():
        if tag == 'PUNCT':
            punct_count += 1
        
    return punct_count/total_pos

In [9]:
# extract relevant column as series
pos_series = toefl_tagged['POS'].astype(str)

# apply function
toefl_tagged['prop_punct'] = pos_series.apply(lambda x: prop_punct(x))

In [10]:
toefl_tagged.head()

,Unnamed: 0,Filename,text,Language,Proficiency,POS,DEP,NER,prop_punct
0,0,88.txt,"Some people might think that traveling in a group led by a tour guide is a good way. But, a group tour normally has its strict schedule and does not offer the tourists in group any chance to meet many people in new place. \nSo, I strongly disagree with the statement that the best way to travel is in a group led by a tour guide. \n\nFirst, a group tour has its own strict schedule and all the tour team members should follow it. And it means that I can not choose the place where I want go and the food which I want to eat. For example, when I went to Thailand for the first time, I chose a group tour because I was young and it was the first time to go abroad. I really wanted to go many attractive shops and spots in the street in Bangkok, but I couldn't go there because I was in a group tour and I had to follow the schedule which the tour company offered. And I had to wake up early in the morning and had to go back to hotel in early evening, even though I wanted to enjoy more sleep in the bed in the morning and wanted to go outside in the night. It was really bad experience. \n\nSecond, a group tour does not offer many opportunities to meet other people for its tour team members. Becuase a group tour moves together by its tour bus, it is difficult to meet many people in new places. But if I travel alone, I can meet many people and feel their cultures. When I went to Tokyo by myself, I met really many people there. I could talk with them and some of them became my friends! Even though I could not go a lot of sightseeing spots as efficiently as a group tour, choosing a place where I go and going there by a subway were really exciting experience and Japanese people were kind to let me know the ways. \n\nAll things considered, I definitely belive that a group tour led by a tour guide is not interesting or exciting at all. By going new place alone, we can learn more about the new place and feel much about the people there. \n",KOR,high,DET NOUN VERB VERB SCONJ VERB ADP DET NOUN VERB ADP DET NOUN NOUN AUX DET ADJ NOUN PUNCT CCONJ PUNCT DET NOUN NOUN ADV AUX DET ADJ NOUN CCONJ AUX PART VERB DET NOUN ADP NOUN DET NOUN PART VERB ADJ NOUN ADP ADJ NOUN PUNCT SPACE ADV PUNCT PRON ADV VERB ADP DET NOUN SCONJ DET ADJ NOUN PART VERB AUX ADP DET NOUN VERB ADP DET NOUN NOUN PUNCT SPACE ADV PUNCT DET NOUN NOUN AUX DET ADJ ADJ NOUN CCONJ DET DET NOUN NOUN NOUN VERB VERB PRON PUNCT CCONJ PRON VERB SCONJ PRON VERB PART VERB DET NOUN ADV PRON VERB VERB CCONJ DET NOUN DET PRON VERB PART VERB PUNCT ADP NOUN PUNCT ADV PRON VERB ADP PROPN ADP DET ADJ NOUN PUNCT PRON VERB DET NOUN NOUN SCONJ PRON AUX ADJ CCONJ PRON AUX DET ADJ NOUN PART VERB ADV PUNCT PRON ADV VERB PART VERB ADJ ADJ NOUN CCONJ NOUN ADP DET NOUN ADP PROPN PUNCT CCONJ PRON VERB PART VERB ADV SCONJ PRON AUX ADP DET NOUN NOUN CCONJ PRON AUX PART VERB DET NOUN DET DET NOUN NOUN VERB PUNCT CCONJ PRON AUX PART VERB ADP ADV ADP DET NOUN CCONJ AUX PART VERB ADV ADP NOUN ADP ADJ NOUN PUNCT ADV SCONJ PRON VERB PART VERB ADJ NOUN ADP DET NOUN ADP DET NOUN CCONJ VERB PART VERB ADV ADP DET NOUN PUNCT PRON AUX ADV ADJ NOUN PUNCT SPACE ADV PUNCT DET NOUN NOUN AUX PART VERB ADJ NOUN PART VERB ADJ NOUN ADP DET NOUN NOUN NOUN PUNCT NOUN DET NOUN NOUN VERB ADV ADP DET NOUN NOUN PUNCT PRON AUX ADJ PART VERB ADJ NOUN ADP ADJ NOUN PUNCT CCONJ SCONJ PRON VERB ADV PUNCT PRON VERB VERB ADJ NOUN CCONJ VERB DET NOUN PUNCT ADV PRON VERB ADP PROPN ADP PRON PUNCT PRON VERB ADV ADJ NOUN ADV PUNCT PRON VERB VERB ADP PRON CCONJ DET ADP PRON VERB DET NOUN PUNCT ADV SCONJ PRON VERB PART VERB DET NOUN ADP NOUN NOUN ADV ADV SCONJ DET NOUN NOUN PUNCT VERB DET NOUN ADV PRON VERB CCONJ VERB ADV ADP DET NOUN AUX ADV ADJ NOUN CCONJ ADJ NOUN AUX ADJ PART VERB PRON VERB DET NOUN PUNCT SPACE DET NOUN VERB PUNCT PRON ADV VERB SCONJ DET NOUN NOUN VERB ADP DET NOUN NOUN AUX PART ADJ CCONJ ADJ ADV ADV PUNCT ADP VERB ADJ NOUN ADV PUNCT PRON VERB VERB ADJ ADP DET

#### Average sentence length

In [11]:
# create function
def avg_sentlength(series):
    """Calculates the average sentence lengths of a text"""

    sents = re.split('[\.|\?|\!]+', series)
    lengths = []
    
    for s in sents:
        s = s.strip().split()
        length = len(s)
        lengths.append(length)
        
    return mean(lengths)

In [15]:
# extract relevant column as series
text_series = toefl_tagged['text'].astype(str)

# apply function
toefl_tagged['avg_sentlength'] = text_series.apply(lambda x: avg_sentlength(x))

In [16]:
toefl_tagged.head(3)

,Unnamed: 0,Filename,text,Language,Proficiency,POS,DEP,NER,prop_punct,avg_sentlength
0,0,88.txt,"Some people might think that traveling in a group led by a tour guide is a good way. But, a group tour normally has its strict schedule and does not offer the tourists in group any chance to meet many people in new place. \nSo, I strongly disagree with the statement that the best way to travel is in a group led by a tour guide. \n\nFirst, a group tour has its own strict schedule and all the tour team members should follow it. And it means that I can not choose the place where I want go and the food which I want to eat. For example, when I went to Thailand for the first time, I chose a group tour because I was young and it was the first time to go abroad. I really wanted to go many attractive shops and spots in the street in Bangkok, but I couldn't go there because I was in a group tour and I had to follow the schedule which the tour company offered. And I had to wake up early in the morning and had to go back to hotel in early evening, even though I wanted to enjoy more sleep in the bed in the morning and wanted to go outside in the night. It was really bad experience. \n\nSecond, a group tour does not offer many opportunities to meet other people for its tour team members. Becuase a group tour moves together by its tour bus, it is difficult to meet many people in new places. But if I travel alone, I can meet many people and feel their cultures. When I went to Tokyo by myself, I met really many people there. I could talk with them and some of them became my friends! Even though I could not go a lot of sightseeing spots as efficiently as a group tour, choosing a place where I go and going there by a subway were really exciting experience and Japanese people were kind to let me know the ways. \n\nAll things considered, I definitely belive that a group tour led by a tour guide is not interesting or exciting at all. By going new place alone, we can learn more about the new place and feel much about the people there. \n",KOR,high,DET NOUN VERB VERB SCONJ VERB ADP DET NOUN VERB ADP DET NOUN NOUN AUX DET ADJ NOUN PUNCT CCONJ PUNCT DET NOUN NOUN ADV AUX DET ADJ NOUN CCONJ AUX PART VERB DET NOUN ADP NOUN DET NOUN PART VERB ADJ NOUN ADP ADJ NOUN PUNCT SPACE ADV PUNCT PRON ADV VERB ADP DET NOUN SCONJ DET ADJ NOUN PART VERB AUX ADP DET NOUN VERB ADP DET NOUN NOUN PUNCT SPACE ADV PUNCT DET NOUN NOUN AUX DET ADJ ADJ NOUN CCONJ DET DET NOUN NOUN NOUN VERB VERB PRON PUNCT CCONJ PRON VERB SCONJ PRON VERB PART VERB DET NOUN ADV PRON VERB VERB CCONJ DET NOUN DET PRON VERB PART VERB PUNCT ADP NOUN PUNCT ADV PRON VERB ADP PROPN ADP DET ADJ NOUN PUNCT PRON VERB DET NOUN NOUN SCONJ PRON AUX ADJ CCONJ PRON AUX DET ADJ NOUN PART VERB ADV PUNCT PRON ADV VERB PART VERB ADJ ADJ NOUN CCONJ NOUN ADP DET NOUN ADP PROPN PUNCT CCONJ PRON VERB PART VERB ADV SCONJ PRON AUX ADP DET NOUN NOUN CCONJ PRON AUX PART VERB DET NOUN DET DET NOUN NOUN VERB PUNCT CCONJ PRON AUX PART VERB ADP ADV ADP DET NOUN CCONJ AUX PART VERB ADV ADP NOUN ADP ADJ NOUN PUNCT ADV SCONJ PRON VERB PART VERB ADJ NOUN ADP DET NOUN ADP DET NOUN CCONJ VERB PART VERB ADV ADP DET NOUN PUNCT PRON AUX ADV ADJ NOUN PUNCT SPACE ADV PUNCT DET NOUN NOUN AUX PART VERB ADJ NOUN PART VERB ADJ NOUN ADP DET NOUN NOUN NOUN PUNCT NOUN DET NOUN NOUN VERB ADV ADP DET NOUN NOUN PUNCT PRON AUX ADJ PART VERB ADJ NOUN ADP ADJ NOUN PUNCT CCONJ SCONJ PRON VERB ADV PUNCT PRON VERB VERB ADJ NOUN CCONJ VERB DET NOUN PUNCT ADV PRON VERB ADP PROPN ADP PRON PUNCT PRON VERB ADV ADJ NOUN ADV PUNCT PRON VERB VERB ADP PRON CCONJ DET ADP PRON VERB DET NOUN PUNCT ADV SCONJ PRON VERB PART VERB DET NOUN ADP NOUN NOUN ADV ADV SCONJ DET NOUN NOUN PUNCT VERB DET NOUN ADV PRON VERB CCONJ VERB ADV ADP DET NOUN AUX ADV ADJ NOUN CCONJ ADJ NOUN AUX ADJ PART VERB PRON VERB DET NOUN PUNCT SPACE DET NOUN VERB PUNCT PRON ADV VERB SCONJ DET NOUN NOUN VERB ADP DET NOUN NOUN AUX PART ADJ CCONJ ADJ ADV ADV PUNCT ADP VERB ADJ NOUN ADV PUNCT PRON VERB V

#### Proportion of capitalized words

In [12]:
# create function
def count_capwords(series):
    """Returns the proportion of capitalized words in a text"""
    
    words = series.split()
    cap_count = 0
    
    for w in words:
        if w[0].isupper():
            cap_count += 1
            
    return cap_count/len(words)

In [17]:
# extract relevant column as series
text_series = toefl_tagged['text'].astype(str)

# apply function
toefl_tagged['prop_capwords'] = text_series.apply(lambda x: count_capwords(x))

In [19]:
toefl_tagged.head(3)

,Unnamed: 0,Filename,text,Language,Proficiency,POS,DEP,NER,prop_punct,avg_sentlength,prop_capwords
0,0,88.txt,"Some people might think that traveling in a group led by a tour guide is a good way. But, a group tour normally has its strict schedule and does not offer the tourists in group any chance to meet many people in new place. \nSo, I strongly disagree with the statement that the best way to travel is in a group led by a tour guide. \n\nFirst, a group tour has its own strict schedule and all the tour team members should follow it. And it means that I can not choose the place where I want go and the food which I want to eat. For example, when I went to Thailand for the first time, I chose a group tour because I was young and it was the first time to go abroad. I really wanted to go many attractive shops and spots in the street in Bangkok, but I couldn't go there because I was in a group tour and I had to follow the schedule which the tour company offered. And I had to wake up early in the morning and had to go back to hotel in early evening, even though I wanted to enjoy more sleep in the bed in the morning and wanted to go outside in the night. It was really bad experience. \n\nSecond, a group tour does not offer many opportunities to meet other people for its tour team members. Becuase a group tour moves together by its tour bus, it is difficult to meet many people in new places. But if I travel alone, I can meet many people and feel their cultures. When I went to Tokyo by myself, I met really many people there. I could talk with them and some of them became my friends! Even though I could not go a lot of sightseeing spots as efficiently as a group tour, choosing a place where I go and going there by a subway were really exciting experience and Japanese people were kind to let me know the ways. \n\nAll things considered, I definitely belive that a group tour led by a tour guide is not interesting or exciting at all. By going new place alone, we can learn more about the new place and feel much about the people there. \n",KOR,high,DET NOUN VERB VERB SCONJ VERB ADP DET NOUN VERB ADP DET NOUN NOUN AUX DET ADJ NOUN PUNCT CCONJ PUNCT DET NOUN NOUN ADV AUX DET ADJ NOUN CCONJ AUX PART VERB DET NOUN ADP NOUN DET NOUN PART VERB ADJ NOUN ADP ADJ NOUN PUNCT SPACE ADV PUNCT PRON ADV VERB ADP DET NOUN SCONJ DET ADJ NOUN PART VERB AUX ADP DET NOUN VERB ADP DET NOUN NOUN PUNCT SPACE ADV PUNCT DET NOUN NOUN AUX DET ADJ ADJ NOUN CCONJ DET DET NOUN NOUN NOUN VERB VERB PRON PUNCT CCONJ PRON VERB SCONJ PRON VERB PART VERB DET NOUN ADV PRON VERB VERB CCONJ DET NOUN DET PRON VERB PART VERB PUNCT ADP NOUN PUNCT ADV PRON VERB ADP PROPN ADP DET ADJ NOUN PUNCT PRON VERB DET NOUN NOUN SCONJ PRON AUX ADJ CCONJ PRON AUX DET ADJ NOUN PART VERB ADV PUNCT PRON ADV VERB PART VERB ADJ ADJ NOUN CCONJ NOUN ADP DET NOUN ADP PROPN PUNCT CCONJ PRON VERB PART VERB ADV SCONJ PRON AUX ADP DET NOUN NOUN CCONJ PRON AUX PART VERB DET NOUN DET DET NOUN NOUN VERB PUNCT CCONJ PRON AUX PART VERB ADP ADV ADP DET NOUN CCONJ AUX PART VERB ADV ADP NOUN ADP ADJ NOUN PUNCT ADV SCONJ PRON VERB PART VERB ADJ NOUN ADP DET NOUN ADP DET NOUN CCONJ VERB PART VERB ADV ADP DET NOUN PUNCT PRON AUX ADV ADJ NOUN PUNCT SPACE ADV PUNCT DET NOUN NOUN AUX PART VERB ADJ NOUN PART VERB ADJ NOUN ADP DET NOUN NOUN NOUN PUNCT NOUN DET NOUN NOUN VERB ADV ADP DET NOUN NOUN PUNCT PRON AUX ADJ PART VERB ADJ NOUN ADP ADJ NOUN PUNCT CCONJ SCONJ PRON VERB ADV PUNCT PRON VERB VERB ADJ NOUN CCONJ VERB DET NOUN PUNCT ADV PRON VERB ADP PROPN ADP PRON PUNCT PRON VERB ADV ADJ NOUN ADV PUNCT PRON VERB VERB ADP PRON CCONJ DET ADP PRON VERB DET NOUN PUNCT ADV SCONJ PRON VERB PART VERB DET NOUN ADP NOUN NOUN ADV ADV SCONJ DET NOUN NOUN PUNCT VERB DET NOUN ADV PRON VERB CCONJ VERB ADV ADP DET NOUN AUX ADV ADJ NOUN CCONJ ADJ NOUN AUX ADJ PART VERB PRON VERB DET NOUN PUNCT SPACE DET NOUN VERB PUNCT PRON ADV VERB SCONJ DET NOUN NOUN VERB ADP DET NOUN NOUN AUX PART ADJ CCONJ ADJ ADV ADV PUNCT ADP VERB ADJ NOUN ADV PUN

#### Capitalized words at beginning of sentences

In [18]:
# create function
def prop_capwords_beginningsent(series):
    """Calculates the proportion of capitalized words at the beginning of a sentence"""
    
    sents = re.split('[\.|\?|\!]+', series)

    upper_count = 0
    lower_count = 0

    try:
        for s in sents:
            s = s.strip()
        
            if s[0].isupper():
                upper_count += 1
            elif s[0].islower():
                lower_count += 1
            
    except IndexError:
        pass

    total_sents = upper_count + lower_count
    try:
        prop_upper = upper_count / total_sents
        return prop_upper
    except ZeroDivisionError:
        return upper_count / 1

In [ ]:
# extract relevant column as series
text_series = toefl_tagged['text'].astype(str)

# apply function
toefl_tagged['prop_capwords_beginningsent'] = text_series.apply(lambda x: prop_capwords_beginningsent(x))

In [20]:
toefl_tagged.head(3)

,Unnamed: 0,Filename,text,Language,Proficiency,POS,DEP,NER,prop_punct,avg_sentlength,prop_capwords
0,0,88.txt,"Some people might think that traveling in a group led by a tour guide is a good way. But, a group tour normally has its strict schedule and does not offer the tourists in group any chance to meet many people in new place. \nSo, I strongly disagree with the statement that the best way to travel is in a group led by a tour guide. \n\nFirst, a group tour has its own strict schedule and all the tour team members should follow it. And it means that I can not choose the place where I want go and the food which I want to eat. For example, when I went to Thailand for the first time, I chose a group tour because I was young and it was the first time to go abroad. I really wanted to go many attractive shops and spots in the street in Bangkok, but I couldn't go there because I was in a group tour and I had to follow the schedule which the tour company offered. And I had to wake up early in the morning and had to go back to hotel in early evening, even though I wanted to enjoy more sleep in the bed in the morning and wanted to go outside in the night. It was really bad experience. \n\nSecond, a group tour does not offer many opportunities to meet other people for its tour team members. Becuase a group tour moves together by its tour bus, it is difficult to meet many people in new places. But if I travel alone, I can meet many people and feel their cultures. When I went to Tokyo by myself, I met really many people there. I could talk with them and some of them became my friends! Even though I could not go a lot of sightseeing spots as efficiently as a group tour, choosing a place where I go and going there by a subway were really exciting experience and Japanese people were kind to let me know the ways. \n\nAll things considered, I definitely belive that a group tour led by a tour guide is not interesting or exciting at all. By going new place alone, we can learn more about the new place and feel much about the people there. \n",KOR,high,DET NOUN VERB VERB SCONJ VERB ADP DET NOUN VERB ADP DET NOUN NOUN AUX DET ADJ NOUN PUNCT CCONJ PUNCT DET NOUN NOUN ADV AUX DET ADJ NOUN CCONJ AUX PART VERB DET NOUN ADP NOUN DET NOUN PART VERB ADJ NOUN ADP ADJ NOUN PUNCT SPACE ADV PUNCT PRON ADV VERB ADP DET NOUN SCONJ DET ADJ NOUN PART VERB AUX ADP DET NOUN VERB ADP DET NOUN NOUN PUNCT SPACE ADV PUNCT DET NOUN NOUN AUX DET ADJ ADJ NOUN CCONJ DET DET NOUN NOUN NOUN VERB VERB PRON PUNCT CCONJ PRON VERB SCONJ PRON VERB PART VERB DET NOUN ADV PRON VERB VERB CCONJ DET NOUN DET PRON VERB PART VERB PUNCT ADP NOUN PUNCT ADV PRON VERB ADP PROPN ADP DET ADJ NOUN PUNCT PRON VERB DET NOUN NOUN SCONJ PRON AUX ADJ CCONJ PRON AUX DET ADJ NOUN PART VERB ADV PUNCT PRON ADV VERB PART VERB ADJ ADJ NOUN CCONJ NOUN ADP DET NOUN ADP PROPN PUNCT CCONJ PRON VERB PART VERB ADV SCONJ PRON AUX ADP DET NOUN NOUN CCONJ PRON AUX PART VERB DET NOUN DET DET NOUN NOUN VERB PUNCT CCONJ PRON AUX PART VERB ADP ADV ADP DET NOUN CCONJ AUX PART VERB ADV ADP NOUN ADP ADJ NOUN PUNCT ADV SCONJ PRON VERB PART VERB ADJ NOUN ADP DET NOUN ADP DET NOUN CCONJ VERB PART VERB ADV ADP DET NOUN PUNCT PRON AUX ADV ADJ NOUN PUNCT SPACE ADV PUNCT DET NOUN NOUN AUX PART VERB ADJ NOUN PART VERB ADJ NOUN ADP DET NOUN NOUN NOUN PUNCT NOUN DET NOUN NOUN VERB ADV ADP DET NOUN NOUN PUNCT PRON AUX ADJ PART VERB ADJ NOUN ADP ADJ NOUN PUNCT CCONJ SCONJ PRON VERB ADV PUNCT PRON VERB VERB ADJ NOUN CCONJ VERB DET NOUN PUNCT ADV PRON VERB ADP PROPN ADP PRON PUNCT PRON VERB ADV ADJ NOUN ADV PUNCT PRON VERB VERB ADP PRON CCONJ DET ADP PRON VERB DET NOUN PUNCT ADV SCONJ PRON VERB PART VERB DET NOUN ADP NOUN NOUN ADV ADV SCONJ DET NOUN NOUN PUNCT VERB DET NOUN ADV PRON VERB CCONJ VERB ADV ADP DET NOUN AUX ADV ADJ NOUN CCONJ ADJ NOUN AUX ADJ PART VERB PRON VERB DET NOUN PUNCT SPACE DET NOUN VERB PUNCT PRON ADV VERB SCONJ DET NOUN NOUN VERB ADP DET NOUN NOUN AUX PART ADJ CCONJ ADJ ADV ADV PUNCT ADP VERB ADJ NOUN ADV PUN

#### Proportion of capitalized 'I'

In [24]:
# create function
def prop_capI(series):
    """Calculates the proportion of capitalized 'I' vs 'i'"""
    
    words = series.split()
    i_count = 0
    i_upper = 0

    for w in words:
        if w == 'i':
            i_count += 1
        elif w == 'I':
            i_count += 1
            i_upper +=1
    try:
        i_upper_prop = i_upper / i_count
        return i_upper_prop
    except ZeroDivisionError:
        return -1 # was intially None but value cannot be NaN for classifier

In [25]:
# extract relevant column as series
text_series = toefl_tagged['text'].astype(str)

# apply function
toefl_tagged['prop_capI'] = text_series.apply(lambda x: prop_capI(x))

In [27]:
toefl_tagged.head()

,Unnamed: 0,Filename,text,Language,Proficiency,POS,DEP,NER,prop_punct,avg_sentlength,prop_capwords,prop_capI
0,0,88.txt,"Some people might think that traveling in a group led by a tour guide is a good way. But, a group tour normally has its strict schedule and does not offer the tourists in group any chance to meet many people in new place. \nSo, I strongly disagree with the statement that the best way to travel is in a group led by a tour guide. \n\nFirst, a group tour has its own strict schedule and all the tour team members should follow it. And it means that I can not choose the place where I want go and the food which I want to eat. For example, when I went to Thailand for the first time, I chose a group tour because I was young and it was the first time to go abroad. I really wanted to go many attractive shops and spots in the street in Bangkok, but I couldn't go there because I was in a group tour and I had to follow the schedule which the tour company offered. And I had to wake up early in the morning and had to go back to hotel in early evening, even though I wanted to enjoy more sleep in the bed in the morning and wanted to go outside in the night. It was really bad experience. \n\nSecond, a group tour does not offer many opportunities to meet other people for its tour team members. Becuase a group tour moves together by its tour bus, it is difficult to meet many people in new places. But if I travel alone, I can meet many people and feel their cultures. When I went to Tokyo by myself, I met really many people there. I could talk with them and some of them became my friends! Even though I could not go a lot of sightseeing spots as efficiently as a group tour, choosing a place where I go and going there by a subway were really exciting experience and Japanese people were kind to let me know the ways. \n\nAll things considered, I definitely belive that a group tour led by a tour guide is not interesting or exciting at all. By going new place alone, we can learn more about the new place and feel much about the people there. \n",KOR,high,DET NOUN VERB VERB SCONJ VERB ADP DET NOUN VERB ADP DET NOUN NOUN AUX DET ADJ NOUN PUNCT CCONJ PUNCT DET NOUN NOUN ADV AUX DET ADJ NOUN CCONJ AUX PART VERB DET NOUN ADP NOUN DET NOUN PART VERB ADJ NOUN ADP ADJ NOUN PUNCT SPACE ADV PUNCT PRON ADV VERB ADP DET NOUN SCONJ DET ADJ NOUN PART VERB AUX ADP DET NOUN VERB ADP DET NOUN NOUN PUNCT SPACE ADV PUNCT DET NOUN NOUN AUX DET ADJ ADJ NOUN CCONJ DET DET NOUN NOUN NOUN VERB VERB PRON PUNCT CCONJ PRON VERB SCONJ PRON VERB PART VERB DET NOUN ADV PRON VERB VERB CCONJ DET NOUN DET PRON VERB PART VERB PUNCT ADP NOUN PUNCT ADV PRON VERB ADP PROPN ADP DET ADJ NOUN PUNCT PRON VERB DET NOUN NOUN SCONJ PRON AUX ADJ CCONJ PRON AUX DET ADJ NOUN PART VERB ADV PUNCT PRON ADV VERB PART VERB ADJ ADJ NOUN CCONJ NOUN ADP DET NOUN ADP PROPN PUNCT CCONJ PRON VERB PART VERB ADV SCONJ PRON AUX ADP DET NOUN NOUN CCONJ PRON AUX PART VERB DET NOUN DET DET NOUN NOUN VERB PUNCT CCONJ PRON AUX PART VERB ADP ADV ADP DET NOUN CCONJ AUX PART VERB ADV ADP NOUN ADP ADJ NOUN PUNCT ADV SCONJ PRON VERB PART VERB ADJ NOUN ADP DET NOUN ADP DET NOUN CCONJ VERB PART VERB ADV ADP DET NOUN PUNCT PRON AUX ADV ADJ NOUN PUNCT SPACE ADV PUNCT DET NOUN NOUN AUX PART VERB ADJ NOUN PART VERB ADJ NOUN ADP DET NOUN NOUN NOUN PUNCT NOUN DET NOUN NOUN VERB ADV ADP DET NOUN NOUN PUNCT PRON AUX ADJ PART VERB ADJ NOUN ADP ADJ NOUN PUNCT CCONJ SCONJ PRON VERB ADV PUNCT PRON VERB VERB ADJ NOUN CCONJ VERB DET NOUN PUNCT ADV PRON VERB ADP PROPN ADP PRON PUNCT PRON VERB ADV ADJ NOUN ADV PUNCT PRON VERB VERB ADP PRON CCONJ DET ADP PRON VERB DET NOUN PUNCT ADV SCONJ PRON VERB PART VERB DET NOUN ADP NOUN NOUN ADV ADV SCONJ DET NOUN NOUN PUNCT VERB DET NOUN ADV PRON VERB CCONJ VERB ADV ADP DET NOUN AUX ADV ADJ NOUN CCONJ ADJ NOUN AUX ADJ PART VERB PRON VERB DET NOUN PUNCT SPACE DET NOUN VERB PUNCT PRON ADV VERB SCONJ DET NOUN NOUN VERB ADP DET NOUN NOUN AUX PART ADJ CCONJ ADJ ADV ADV PUNCT ADP VERB ADJ NO

#### Save features to new csv file

In [ ]:
nli_tagged = nli_tagged.drop("NER", axis=1)

In [ ]:
nli_tagged

,fileno,L1,written_text,POS,DEP,prop_punct,avg_sentlength,prop_capwords,prop_capwords_beginningsent,prop_capI
0,10017,ARA,"life without concepts is meaing of death. it is very important for students to discover the world around them by learining true concepts.There are three common benifets of that . first is to gaive chance to have apece world since there is common agreement on that concepts . we have notce that after the foundiation of the international orgnization in many aspects such as,political,health and social the quality of life has been improvedsecound it improve sentific movment since it has bult on the base of ideas and knowledfinally it is protection of the waild redical ideas which control the socites if there is no fair jujmentin my opinion facts will be result of what we belive on",NOUN ADP SPACE NOUN SPACE AUX VERB ADP SPACE NOUN PUNCT PRON AUX ADV ADJ ADP NOUN PART VERB DET NOUN ADP PRON ADP VERB SPACE ADJ NOUN PUNCT PRON AUX NUM SPACE ADJ NOUN ADP DET PUNCT SPACE ADV SPACE AUX PART VERB NOUN PART AUX ADJ NOUN SCONJ PRON AUX ADJ NOUN ADP DET NOUN PUNCT PRON AUX PROPN SCONJ ADP DET NOUN ADP DET ADJ NOUN ADP ADJ NOUN SPACE ADJ SCONJ PUNCT ADJ PUNCT NOUN CCONJ ADJ DET NOUN ADP NOUN AUX AUX VERB PRON VERB ADJ NOUN SCONJ PRON AUX NOUN ADP DET NOUN SPACE ADP NOUN CCONJ ADV PRON AUX NOUN ADP DET ADJ SPACE ADJ NOUN DET VERB DET NOUN SCONJ PRON AUX DET ADJ NOUN DET NOUN NOUN VERB AUX NOUN ADP PRON PRON VERB ADP,nsubj prep pobj aux ROOT prep pobj punct nsubj ROOT advmod acomp mark nsubj aux advcl det dobj prep pobj prep pcomp amod dobj punct expl ROOT nummod amod attr prep pobj punct advmod ROOT aux xcomp dobj aux acl compound dobj mark expl advcl amod attr prep det pobj punct nsubj ROOT dobj mark prep det pobj prep det amod pobj prep amod pobj amod prep punct amod punct conj cc conj det pobj prep pobj aux advcl acomp nsubj ccomp amod dobj mark nsubj advcl dobj prep det pobj prep pobj cc advmod nsubj conj attr prep det nmod amod pobj nsubj relcl det dobj mark expl advcl det amod attr poss compound nsubj aux ROOT attr prep pobj nsubj pcomp prep,0.044776,23.200000,0.000000,0.200000,-1.000000
1,10018,ARA,"Nowadays, knowledge has changed, methods of approach are different, concepts has become more difficut to master, students have different abilities, there are students with limited abilites and stretched to students with overwhelming abilities, this does not put an end to learning experience, as there is a question that is floating in our minds and leads a person to a deeper thinking and makes him put his knowlege behind him. Is it more important for students to understand ideas and concepts than it is to learn facts ?To begin with, I surley accomodate this, and it is more influencial for students to understand ideas and concepts than to learn facts, for example: what would it be better if a student knows that the sea is deep, or learns why its deep, that will give student a deeper, wider, and a variety of ways to think, students will be more efficient in their life,thus leading to better study and a safer vision to their future, and start thinking in a convergantly more significant way.In the past, people had limited recources, no computers, no television, and it would be good if they had books with a quite limited explanation, so our past ansistors depended on their logic, religon, and the scenario that they pass through everyday to learn facts, which lead them to a succesful life that we live nowadays, inventions and innovations passed through, so what is left?Its more important to understand ideas and concepts, as I have mentioned that all facts have been produced, so what is left for a student nowadays is to dive deeper in ideas and concepts and try to get out with a sucessful summary at the end of the day, because one day he will be tested with the facts in front of him. Example: in a math exam a student gets all rules that he might need to solve the qustion, but what he needs is the concept of how to solve the ques

In [29]:
toefl_tagged.to_csv("/content/drive/MyDrive/NLP Shared task/toefl11_trainingdata_features")